## Connecting to ADLS

In [ ]:
# display(dbutils.fs.ls("abfss://processed@ADLS_name.dfs.core.windows.net"))

### Exploring Data
##### Displaying ts_registration dataset

In [ ]:
display(spark.read.csv("abfss://raw@ADLS_name.dfs.core.windows.net/ts_registration.csv",header=True))

### 1) Creating DataFrame

In [ ]:
reg_df = spark.read.csv("abfss://raw@ADLS_name.dfs.core.windows.net/ts_registration.csv",header=True,inferSchema=True,escape='"')

In [ ]:
display(reg_df)

In [ ]:
reg_df.printSchema()

In [ ]:
from pyspark.sql.functions import count,when,isnull,col,desc,trim,coalesce

#### 2) Checking Nulls

In [ ]:
reg_df.select([count(when(isnull(c),c)).alias(c) for c in reg_df.columns]).show(vertical=True)

### 3) Temp view
##### Creating Temp View to write SQL quries

In [ ]:
reg_df.createOrReplaceTempView("ts")

In [ ]:
spark.sql("SELECT * FROM ts").show(5)

In [ ]:
display(spark.sql("SELECT OfficeCd,count(officeCd) as cnt FROM ts GROUP BY officeCd ORDER BY cnt DESC"))

### 5) Removing Duplicates
##### Removing Duplicates of last registrationNo

In [ ]:
te_df = reg_df.orderBy(col("registrationNo").desc()).dropDuplicates(["registrationNo"])

In [ ]:
te_df.count()

In [ ]:
display(te_df.groupBy("fromDate").agg(count('registrationNo')).orderBy("fromDate"))

##### 5.1) Dropping unwanted columns "reserve_no"

In [ ]:
te_df = te_df.drop("reserve_no")

##### 5.2) Dropping unwanted columns "slno"

In [ ]:
te_df = te_df.drop("slno")

In [ ]:
te_df.select([count(when(isnull(c),c)).alias(c) for c in te_df.columns]).show(vertical=True)

#### 5.3) Dropping rows
##### Dropping rows that dosent have "registrationNo"

In [ ]:
display(te_df.filter(col("registrationNo").isNull()))

In [ ]:
te_df = te_df.na.drop(subset=["registrationNo"])

In [ ]:
te_df.select([count(when(isnull(c),c)).alias(c) for c in te_df.columns]).show(vertical=True)

### 6) Checking nulls in fuel

In [ ]:
fuel_df = te_df.filter(col("fuel").isNull())

In [ ]:
fuel_df.count()

In [ ]:
display(fuel_df.groupBy('modelDesc').agg(count('modelDesc')).orderBy(desc("count(modelDesc)")))

###### While searching these modelDesc are trailors of trucks and tractors etc these are additional parts for trucks and tractors  \
###### These can be removed

In [ ]:
drop_df = te_df.dropna(subset=["fuel"])

#### Nulls from the dataset has been cleared

In [ ]:
drop_df.select([count(when(isnull(c),c)).alias(c) for c in drop_df.columns]).show(vertical=True)

#### 7) Data quality checks

In [ ]:
display(drop_df.groupBy('fuel').count().orderBy(desc("count")))

In [ ]:
dq_df = drop_df.withColumn("fuel", when(drop_df.fuel == "Petrol", "PETROL").when(drop_df.fuel == "petrol", "PETROL").otherwise(drop_df.fuel))

In [ ]:
dq_df = dq_df.withColumn("fuel", when(trim(dq_df.fuel) == "DIESEL", "DIESEL").otherwise(dq_df.fuel))

###### These are additional parts of a trucks and tractors and does not require fuel 
###### Its better to remove this

In [ ]:
display(dq_df.filter(dq_df.fuel == 0))

In [ ]:
dq_df = dq_df.filter(dq_df.fuel != '0')

In [ ]:
display(dq_df.filter(dq_df.fuel == -1))

##### Removing rows

In [ ]:
dq_df = dq_df.filter(dq_df.fuel != '-1')

In [ ]:
display(dq_df.groupBy('fuel').count().orderBy(desc("count")))

In [ ]:
final_df = dq_df

In [ ]:
display(final_df)

#### Write df to Parquet format

In [ ]:
final_df.coalesce(1).write.parquet("abfss://processed@ADLS_name.dfs.core.windows.net/tsreg")

In [ ]:
par_df = spark.read.parquet("abfss://processed@ADLS_name.dfs.core.windows.net/tsreg")

In [ ]:
par_df.count()